# NMF Topic Modeling

Use BII, 20 cores, 3 hours, 100 Ram.

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time

from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models.coherencemodel import CoherenceModel
import ast

In [2]:
#TODO:Input a dataframe that are ai related abstracts, need variables: final_frqwds_removed
#abstracts = pd.read_csv(r'/home/zz3hs/git/dspg21RnD/data/dspg21RnD/bert_ai_abstracts.csv')   

In [3]:
abstracts["final_frqwds_removed"]

0        ['effectiveness', 'computerize', 'instructiona...
1        ['direct', 'identification', 'basic', 'apply',...
2        ['generation', 'economist', 'harrod', 'domar',...
3        ['developer', 'create', 'prototype', 'playing_...
4        ['team', 'prototype', 'gowrite', 'platform', '...
                               ...                        
22553    ['public', 'health', 'child', 'mental', 'healt...
22554    ['purpose', 'advance', 'jacket_gpu_engine', 'm...
22555    ['american_indian', 'science', 'engineering', ...
22556    ['astep', 'observational', 'technology', 'enab...
22557    ['advance', 'genome', 'interactive', 'explorat...
Name: final_frqwds_removed, Length: 22558, dtype: object

In [24]:
#pd.set_option("display.max_colwidth", -1)
#abstracts[abstracts["cosine_similarity_score"] >0.55].ABSTRACT

In [25]:
#abstracts[abstracts["cosine_similarity_score"]  <0.41].ABSTRACT

# Coherence Model to find the optimal number of topics for NMF

In [41]:
docs = abstracts["final_frqwds_removed"]

clean_docs = []
for doc in docs:
    doc = ast.literal_eval(doc)
    clean_docs.append(doc)
    
text = []
for abstract in clean_docs:
    text.append(" ".join(abstract))

In [21]:
clean_docs[0][0]

'effectiveness'

In [43]:
text[0]

'effectiveness computerize instructional adjunct standard integrate brain injury rehabilitation member satisfaction utilize computerize inform participant instructional selection maximize benefit'

In [12]:
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=3)
tf_idf = tfidf_vectorizer.fit_transform(text)

In [16]:
tf_idf.shape

(22558, 23678)

In [22]:
def createVars(docs):

    # Create the variables needed for NMF from df[final_frqwds_removed]: dictionary (id2word), corpus
    
    # Create Dictionary
    id2word = gensim.corpora.Dictionary(docs)

    if len(docs) <= 100000:
        id2word.filter_extremes(no_below=3, no_above=1.0,  keep_n = 100000)
        print("Use keep_n = 100,000 defalt.")
    else:
        id2word.filter_extremes(no_below=3, no_above=1.0,  keep_n = len(docs))
        print("Number of documents exceed the dafalt number of 100,000. Use the keep_n = number of document.")

    # Create Corpus (Term Document Frequency)

    #Creates a count for each unique word appearing in the document, where the word_id is substituted for the word
    corpus = [id2word.doc2bow(doc) for doc in docs]

    return id2word, corpus

In [23]:
#TODO: run the following code to generate id2word and corpus
#id2word, corpus = createVars(clean_docs)
#TODO: RENAME the file, run the code to save the output
docs  = clean_docs
#pickle.dump([corpus, id2word, docs], open('../../data/dspg21RnD/coherence_vars_XXX.sav','wb'))

Use keep_n = 100,000 defalt.


# Read in Coherence file

In [3]:
#TODO: Read in your coherence data (change the name of the file)
f = open('../../data/dspg21RnD/coherence_vars_bert.sav', 'rb')
[corpus, id2word, docs] = pickle.load(f)
f.close()

In [4]:
len(docs)

22558

In [5]:
docs[0]

['effectiveness',
 'computerize',
 'instructional',
 'adjunct',
 'standard',
 'integrate',
 'brain',
 'injury',
 'rehabilitation',
 'member',
 'satisfaction',
 'utilize',
 'computerize',
 'inform',
 'participant',
 'instructional',
 'selection',
 'maximize',
 'benefit']

In [6]:
text = []
for doc in docs:
    text.append(" ".join(doc))

In [7]:
text[0]

'effectiveness computerize instructional adjunct standard integrate brain injury rehabilitation member satisfaction utilize computerize inform participant instructional selection maximize benefit'

In [8]:
# function slightly modified from https://nlpforhackers.io/topic-modeling/

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)
            
# Function to format topics as a "list of list of strings".
# Needed for topic coherence function in Gensim

# function modified from https://nlpforhackers.io/topic-modeling/

def list_topics(model, vectorizer, top_n=10):

    #input. top_n: how many words to list per topic.  If -1, then list all words.
       
    topic_words = []
    
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
            
        if top_n == -1:   
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[::-1]])
        else:
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n - 1:-1]])
        
    return topic_words

In [9]:
# create document-term matrix - TFIDF 
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=3)
tf_idf = tfidf_vectorizer.fit_transform(text)

In [10]:
# function adapted from https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/

def nmf_metrics(doc_term_matrix, n_topics, vectorizer, corpus, id2word, docs, rand_start):
    """
    Compute c_v topic coherence for various number of topics
    Parameters:
    ----------
    tf_idf
    n_topics : list of number of topics
    Returns:
    -------
    coherence_values : c_v topic coherence values corresponding to the NMF model with respective number of topics
    """
    
    coherence_values = []
    
    i = rand_start
    for num_topics in n_topics:

        # create model
        t1 = time.time()
        nmf_model = NMF(n_components=num_topics, random_state = i)
        nmf_model.fit_transform(doc_term_matrix)
        t2 = time.time()
        print(f"  Model time: {t2-t1}")
        
        # create list of topics
        topics = list_topics(nmf_model, vectorizer, top_n=10)
        
        # calculate coherence
        t1 = time.time()
        
        #TODO:manually adjust number of processes
        cm = CoherenceModel(topics=topics, corpus=corpus, dictionary=id2word, texts=docs, 
                            coherence='c_v', #model for calculating coherence score
                            processes=19 #for smaller corpus, pronesses= number of cores - 5
                           ) #window_size=500 ) 
        coherence_values.append(cm.get_coherence())
        t2 = time.time()
        print(f"  Coherence time: {t2-t1}")
        
        # output completion message
        i = i+1
        print('Number of topics =', num_topics, "complete.")

    return coherence_values


In [ ]:
# code copied from https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/
# minor alterations made
n_topics = list(range(5,51,5)) + [75,100,125] #takes about 4-5 hrs to run

num_runs = 10

col_names = [f"iteration {i}" for i in range(num_runs)]
nmf_c = pd.DataFrame(index = n_topics, columns = col_names)

for i in range(num_runs):
    print(f"Iteration {i}")
    
    # run models
    c = nmf_metrics(doc_term_matrix=tf_idf, n_topics=n_topics, vectorizer=tfidf_vectorizer, 
                         corpus=corpus, id2word=id2word, docs=docs, rand_start = (i)*len(n_topics))
    
    # save results
    nmf_c[f"iteration {i}"] = c

Iteration 0
  Model time: 3.132255792617798
  Coherence time: 2.2182235717773438
Number of topics = 5 complete.
  Model time: 6.946704864501953
  Coherence time: 3.4823157787323
Number of topics = 10 complete.
  Model time: 10.436957359313965
  Coherence time: 4.005496978759766
Number of topics = 15 complete.
  Model time: 10.771311283111572
  Coherence time: 4.4601075649261475
Number of topics = 20 complete.


/home/zz3hs/.conda/envs/crystal_bert/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


  Model time: 30.37483859062195
  Coherence time: 5.9043214321136475
Number of topics = 25 complete.
  Model time: 33.9063618183136
  Coherence time: 6.94676661491394
Number of topics = 30 complete.


In [ ]:
# TODO: save results, change file name
nmf_c.to_pickle("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/nmf_bert_10.pkl")